In [2]:
import pandas as pd

In [3]:
school_complete_load = "C://Users/natha/Documents/BootCamp HW Assigments/Pandas Hw/pandas-challenge/schools_complete.csv"
students_complete_load = "C://Users/natha/Documents/BootCamp HW Assigments/Pandas Hw/pandas-challenge/students_complete.csv"

In [4]:
# importing data to work on
school_data = pd.read_csv(school_complete_load)
student_data = pd.read_csv(students_complete_load)

In [5]:
# merging my two data files to create one object to work on
school_data_merged = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [6]:
# calculating total students by using there Student IDs
total_number_students =  school_data_merged["Student ID"].nunique()
# calculating number students with math and reading scores above 7
j = 0
for i in school_data_merged["math_score"]:
    if i >= 70:
        j = j + 1
# Calculating percentage of students who got 70 and above in math
Math_Pass_Rate = j/total_number_students
j = 0
for i in school_data_merged["reading_score"]:
    if i >= 70:
        j = j + 1
# Calculating percentage of students who got 70 and above in reading
Reading_Pass_Rate = j/total_number_students
# creating dataframe that holds various statistical measures
district_df = pd.DataFrame([{"Total Schools": school_data_merged["school_name"].nunique(), "Total Students": total_number_students, "Total Budget": school_data["budget"].sum(), "Average Math Score": (school_data_merged["math_score"].sum())/total_number_students, "Average Reading Score": (school_data_merged["reading_score"].sum())/total_number_students, "% Passing Math": Math_Pass_Rate*100, "% Passing Reading": Reading_Pass_Rate*100, "% Overall Passing Rate": (((Math_Pass_Rate + Reading_Pass_Rate)/2)*100)}])
# currency formating certain colunms in above dataframe
district_df["Total Budget"] = district_df["Total Budget"].map('${:,.2f}'.format)
district_df["Total Students"] = district_df["Total Students"].map('{:,}'.format)
# reordering my above dataframe to desired look
District_Data = district_df[["Total Schools", "Total Students", "Total Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].round(2)       

In [7]:
# creating grouped dataframe by school
grouped = school_data_merged.groupby("school_name").agg({"type": "first", "Student ID": "count", "budget": "first", "reading_score": "mean", "math_score": "mean"}).rename(columns = {"school_name": "School Name", "type": "School Type", "Student ID": "Total Students", "budget": "Total School Budget", "reading_score": "Average Reading Score", "math_score": "Average Math Score"})      
# adding column to above dataframe
grouped["Per Student Budget"] = grouped["Total School Budget"]/grouped["Total Students"]
# I need an object to manipulate further down.
Spending_Object = grouped["Per Student Budget"] = grouped["Total School Budget"]/grouped["Total Students"]
# currency formating 
grouped["Total School Budget"] = grouped["Total School Budget"].map('${:,.2f}'.format)
grouped["Per Student Budget"] = grouped["Per Student Budget"].map('${:,.2f}'.format)
# I need an object to manipulate further down.
School_Size_Object = grouped["Total Students"]

In [8]:
# creating dataframe object the holds only students with 70 or above in math
Passing_Math = school_data_merged.loc[(school_data_merged["math_score"] >= 70)]
# creating dataframe object the holds only students with 70 or above in reading
Passing_Reading = school_data_merged.loc[(school_data_merged["reading_score"] >= 70)]
# using above math dataframe to find the number of students with scores 70 or above by each school for use later
Passed_Math = Passing_Math.groupby("school_name").agg({"math_score": "count"})
# using above math dataframe to find the number of students with scores 70 or above by each school for use later
Passed_Reading = Passing_Reading.groupby("school_name").agg({"reading_score": "count"})

In [9]:
# adding % passing math column to my school groupby dataframe based on created columns above.
grouped["% Passing Math"] = (Passed_Math["math_score"]/grouped["Total Students"])*100
# adding % passing reading column to my school groupby dataframe based on created columns above.
grouped["% Passing Reading"] = (Passed_Reading["reading_score"]/grouped["Total Students"])*100
# # adding % passing overall column to my school groupby dataframe based on just added columns
grouped["% Overall Passing Rate"] = (grouped["% Passing Math"] + grouped["% Passing Reading"])/2
# currency formating
grouped["Total Students"] = grouped["Total Students"].map('{:,}'.format)

In [10]:
# storing my completed dataframe in variable for use as desired
School_Summary_Data = grouped[["School Type", "Total Students", "Total School Budget" ,"Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].rename_axis("School Name").round(2)

In [11]:
# sorting my completed school groupby dataframe by top five schools based on % overall passing column and storing in variable
Top_Five_Data = (grouped.sort_values("% Overall Passing Rate", ascending = False)).head(n = 5).rename_axis("School Name").round(2)

In [12]:
# sorting my completed school groupby dataframe by bottom five schools based on % overall passing column and storing in variable
Bottom_Five_Data = (grouped.sort_values("% Overall Passing Rate")).head(n = 5).rename_axis("School Name").round(2)

In [13]:
# organizing my data/grouping by grade level now
# creating another dataframe grouped by school name
# needed to add one column in order to complete my endgame, (agg({"type": "first"})). Will drop this column further down the road
Grade_Level_Math = (school_data_merged.groupby("school_name").agg({"type": "first"}))
# creating columns of average math scores by school and by grades 9, 10, 11 ,12 and storing as variables
Grade_9_Math = school_data_merged.loc[(school_data_merged["grade"] == "9th")].groupby("school_name").agg({"math_score": "mean"})
Grade_10_Math = school_data_merged.loc[(school_data_merged["grade"] == "10th")].groupby("school_name").agg({"math_score": "mean"})
Grade_11_Math = school_data_merged.loc[(school_data_merged["grade"] == "11th")].groupby("school_name").agg({"math_score": "mean"})
Grade_12_Math = school_data_merged.loc[(school_data_merged["grade"] == "12th")].groupby("school_name").agg({"math_score": "mean"})
# adding above average math columns to my groupby Grade_Level_Math dataframe
Grade_Level_Math["9th"] = Grade_9_Math["math_score"]
Grade_Level_Math["10th"] = Grade_10_Math["math_score"]
Grade_Level_Math["11th"] = Grade_11_Math["math_score"]
Grade_Level_Math["12th"] = Grade_12_Math["math_score"]
# storing my complete average math dataframe, renaming and adding labels, and dropping the "type" column talked about earlier
Complete_Grade_Level_Math_Data = Grade_Level_Math.drop(columns = "type").rename_axis("School Name").rename_axis("Average Math Scores", axis = "columns").round(2)

In [14]:
# doing everything I did above, but this time to create an average reading score dataframe grouped by school and grade level
Grade_Level_Reading = school_data_merged.groupby("school_name").agg({"type": "first"})
Grade_9_Reading = school_data_merged.loc[(school_data_merged["grade"] == "9th")].groupby("school_name").agg({"reading_score": "mean"})
Grade_10_Reading = school_data_merged.loc[(school_data_merged["grade"] == "10th")].groupby("school_name").agg({"reading_score": "mean"})
Grade_11_Reading = school_data_merged.loc[(school_data_merged["grade"] == "11th")].groupby("school_name").agg({"reading_score": "mean"})
Grade_12_Reading = school_data_merged.loc[(school_data_merged["grade"] == "12th")].groupby("school_name").agg({"reading_score": "mean"})
Grade_Level_Reading["9th"] = Grade_9_Reading["reading_score"]
Grade_Level_Reading["10th"] = Grade_10_Reading["reading_score"]
Grade_Level_Reading["11th"] = Grade_11_Reading["reading_score"]
Grade_Level_Reading["12th"] = Grade_12_Reading["reading_score"]
Complete_Grade_Level_Reading_Data = Grade_Level_Reading.drop(columns = "type").rename_axis("School Name").rename_axis("Average Reading Scores", axis = "columns").round(2)

In [15]:
# creating a new dataframe object based on my above school grouped dataframe, but with exclusion of columns, including the school name.
Scores_By_School = ((grouped[["Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]).reset_index()).drop(columns = "school_name")

In [16]:
# using Spending Object created above based on spending per student and reseting its format so that I can operate on below
Final_Spending_Object = Spending_Object.reset_index()
# creating a dataframe that organizes and groups data by budget spending per student against average scores and passing rates using the the pd.cut and binning method
bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
Scores_By_School["Spending Ranges (per student)"] = pd.cut(Final_Spending_Object[0], bins, labels = group_names)

In [17]:
# creating the final dataframe described above and storing to variable
Scores_By_Spending_Data = Scores_By_School.groupby("Spending Ranges (per student)").agg({"Average Math Score": "mean", "Average Reading Score": "mean", "% Passing Math": "mean", "% Passing Reading": "mean", "% Overall Passing Rate": "mean"}).round(2)      

In [18]:
# using the School_Size_Object created above based on total students per school and reseting its index so as to be able to operate on it
Final_School_Size_Object = School_Size_Object.reset_index()
# creating a dataframe grouped by total students per school to gain a visual of averages and percentage passing based on school size
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
Scores_By_School["School Size"] = pd.cut(Final_School_Size_Object["Total Students"], size_bins, labels = group_names)

In [19]:
# creating final dataframe and storing in variable
Scores_By_School_Size_Data = Scores_By_School.groupby("School Size").agg({"Average Math Score": "mean", "Average Reading Score": "mean", "% Passing Math": "mean", "% Passing Reading": "mean", "% Overall Passing Rate": "mean"}).round(2)

In [20]:
# creating dataframe the descibes summary statistics by school type like district and charter
Scores_By_School_Type = ((grouped[["Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate", "School Type"]]).reset_index()).drop(columns = "school_name")
Scores_By_School_Type_Data = Scores_By_School_Type.groupby("School Type").agg({"Average Math Score": "mean", "Average Reading Score": "mean", "% Passing Math": "mean", "% Passing Reading": "mean", "% Overall Passing Rate": "mean"}).round(2)

In [21]:
# displaying aggregate summary statistcs of schools and students
District_Data

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,80.39


In [22]:
# displaying summary statistics of each individual school
School_Summary_Data

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,74.31
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.59
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.80
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.27
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,73.81
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,94.38
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.64


In [23]:
# displaying top five schools based on % overall passing rate statistic
Top_Five_Data

,School Type,Total Students,Total School Budget,Average Reading Score,Average Math Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",83.98,83.06,$582.00,94.13,97.04,95.59
Thomas High School,Charter,"1,635","$1,043,130.00",83.85,83.42,$638.00,93.27,97.31,95.29
Pena High School,Charter,962,"$585,858.00",84.04,83.84,$609.00,94.59,95.95,95.27
Griffin High School,Charter,"1,468","$917,500.00",83.82,83.35,$625.00,93.39,97.14,95.27
Wilson High School,Charter,"2,283","$1,319,574.00",83.99,83.27,$578.00,93.87,96.54,95.20


In [24]:
# displaying bottom five schools based on % overall passing rate statistic
Bottom_Five_Data 

,School Type,Total Students,Total School Budget,Average Reading Score,Average Math Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",80.74,76.84,$637.00,66.37,80.22,73.29
Figueroa High School,District,"2,949","$1,884,411.00",81.16,76.71,$639.00,65.99,80.74,73.36
Huang High School,District,"2,917","$1,910,635.00",81.18,76.63,$655.00,65.68,81.32,73.50
Johnson High School,District,"4,761","$3,094,650.00",80.97,77.07,$650.00,66.06,81.22,73.64
Ford High School,District,"2,739","$1,763,916.00",80.75,77.10,$644.00,68.31,79.30,73.80


In [25]:
# displaying average math scores by school and by grade level 
Complete_Grade_Level_Math_Data

Average Math Scores,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [26]:
# displaying average reading scores by school and by grade level
Complete_Grade_Level_Reading_Data

Average Reading Scores,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [27]:
# displaying summary statistics based on amount spent per student
Scores_By_Spending_Data 

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (per student),,,,,
<$585,83.46,83.93,93.46,96.61,95.04
$585-615,83.60,83.89,94.23,95.90,95.07
$615-645,79.08,81.89,75.67,86.11,80.89
$645-675,77.00,81.03,66.16,81.13,73.65


In [28]:
# displaying summary statistics based the size of schools
Scores_By_School_Size_Data 

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,94.82
Medium (1000-2000),83.37,83.86,93.60,96.79,95.20
Large (2000-5000),77.75,81.34,69.96,82.77,76.36


In [29]:
# displaying summary statistics based on school type (district vs charter)
Scores_By_School_Type_Data

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,95.10
District,76.96,80.97,66.55,80.80,73.67


In [30]:
Observable_Trends = "I noticed that charter schools outperform district schools based on average reading and math scores and passing rates. I also see that smaller schools have higher reading and math scores on average than larger schools. I also found suprisingly that less money spent per student correlated with higher average reading and math scores/% passing than more spending per student. I would be curious to investigate possible explanations to this relationship."

In [31]:
Observable_Trends

'I noticed that charter schools outperform district schools based on average reading and math scores and passing rates. I also see that smaller schools have higher reading and math scores on average than larger schools. I also found suprisingly that less money spent per student correlated with higher average reading and math scores/% passing than more spending per student. I would be curious to investigate possible explanations to this relationship.'